# **Trees and graphs**
______________________________

# Union-Find Data Structures and Spanning Tree Algorithms

## Contents:
- [Spanning Trees and Minimal Spanning Trees](#Spanning-Trees-and-Minimal-Spanning-Trees)
- [Prim's Algorithm](#Prim's-Algorithm)
- [Kruskal's Algorithm](#Kruskal's-Algorithm)
- [Union-Find Data Structure and Rank Compression](#Union-Find-Data-Structure-and-Rank-Compression)
- [Problem 1](#Problem-1)
- [Problem 2](#Problem-2)

__________
## Spanning Trees and Minimal Spanning Trees

***Concept that works for Weighted Undirected Graphs***

<img src='pics/mst.png' width=300>

**Nodes -> warehouse locations, weights -> transportation costs**


Suppose we need to deliver from 1 to 6. We can route in too many different ways via flooding (broadcasting, etc).
For example, from node 2 we can go to 3, 4 and 5. Then there can be cycles, plus node 6 can receive a parcel from several destinations. 

**So:**

Flooding takes a lot of work (lots of lost bandwidth, lot of traffic).

Need another fast simple decision -> retain enough edges -> graph becomes **acyclic** and **strongly connected** -> ***Spanning Tree***

**Spanning Tree** - a subset of edges S such that a graph of vertices S is a tree

**Minimal Spanning tree** - sum of the edge weigths is as small as possible

<img src='pics/mst2.png' width=300>

Routing through spanning tree is useful

## Prim's Algorithm

***Prim's algorithm is a greedy algorithm for finding shortest paths in a graph based on minimal-spanning-tree generation method***

Prim's algorithm:
1. start from an arbitary root node of a future tree (1)
2. add to a tree an edge to an isolated node with a minimum weight
3. repeat until all the nodes are in the tree

<img src='pics/prim.png' width=600>

Prim's algorithm uses a Min-Priority Queue of the weights

## Kruskal's Algorithm

***Kruskal's algorithm is a greedy algorithm for sorting edges of a graph in ascending order of their weights based on minimal-spanning-tree generation method***

Kruskal's algorithm:
1. imagine/state that initially all the nodes in a given graph are isolated trees in a forest
2. sort the edges in ascending order of weights:
    - for all edges in sorted order:
           - if an edge connects two different trees in a forest:
                   - add it and make them into a single tree
3. finally we have a min-spanning tree

<img src='pics/kruskal.png' width=800>

Complexity - |E|log|E| (E - number of vertices)

To run this algorithm more efficiently we need to use a Union-Find data structure with |v|$\alpha$(|v|) complexity

## Union-Find Data Structure and Rank Compression

#### Family of Disjoint Sets

Sometimes it is useful to have a **Family of Disjoint Sets**

Suppose we have a Universe: $U = \{1, 2, 3, 4, 5, 6\}$  
Family: $S_1:\{1,2\}$, $S_2:\{3, 4, 5\}$, $S_3:\{6\}$ - a collection of subsets of U, which are mutually disjoint $S_1 \cap S_2 = \emptyset$

Every member of this family has a representative element.

Think a given universe as a graph with strongly connected components = family of disjoint subsets, which connect to each other by their representaive elements.

#### Union-Find Data Structure

***This data structure is to represent the disjoint sets***

Disjoint sets may be represented as a forest of trees, where the corresponding set representatives are the roots.

<img src='pics/disjoint_sets.png' width=600>

Union-Find Data Structure allows the following operations:
- __make set__ - take element *j* (which is not yet in a forest), make a singleton set with a self loop around it and add it to a forest
- __find element__ - look for a set that contains an element and return a root (representative of that set))
- __union sets__ - take two elements and merge their sets connecting their representatives creating a new set (tree)) 

New set formed by union, but it may be too long in terms of depth of a tree if sets representatives chosen wrongly.
Total complexity of MakeSet, Union and Find is $\Theta(n) + \Theta(n) + \Theta(m\times n)$

To do it better we use Union by Rank

### Union by rank

***It is a  smart select of a representative***

Rank = property of the root that shows the depth of a tree (max path)

***When union two trees we follow the rule:***

The tree with bigger rank become a representative, if the rank of both trees is the same, then we choose it randomly.

This allows to have nice trees with low depth.

Rank of the final tree would be $\Theta(log(n))$ and Union by Rank operation is $\Theta(n)+\Theta(mlog(n))$ complexity (n = number of make sets, m = number of find operations).

### Rank Compression

***It is a rearranging a tree during a find operation by connecting any node to a root representation***
- firstly we find an element (return its representative) and this operation is 'expensive' (log(n))
- connect an element directly to its representative (some work here)
- next find operation of this element runs in O(1) time

Suppose we make *n* sets, *n* union operations and *m* find operations.

After rank compression instead of nlog(n) we have n$\alpha$(m) complexity, where $\alpha$ is an inverse ackeman function which grows very slow ($\alpha(10^{80} \le 4$)

## Problem 1

Let's complete an implemention of a union-find data structure with rank compression

In [1]:
class DisjointForests:
    def __init__(self, n):
        assert n >= 1, ' Empty disjoint forest is disallowed'
        self.n = n
        self.parents = [None]*n
        self.rank = [None]*n
        
    # Function: dictionary_of_sets
    # Convert the disjoint forest structure into a dictionary d
    # wherein d has an entry for each representative i
    # d[i] maps to each elements which belongs to the tree corresponding to i
    # in the disjoint forest.
    def dictionary_of_sets(self):
        d = {}
        for i in range(self.n):
            if self.is_representative(i):
                d[i] = set([i])
        for j in range(self.n):
            if self.parents[j] != None:
                root = self.find(j)
                assert root in d
                d[root].add(j)
        return d
    
    def make_set(self, j):
        assert 0 <= j < self.n
        assert self.parents[j] == None, 'You are calling make_set on an element multiple times -- not allowed.'
        self.parents[j] = j
        self.rank[j] = 1
        
    def is_representative(self, j):
        return self.parents[j] == j 
    
    def get_rank(self, j):
        return self.rank[j]
    
    # Function: find
    # Implement the find algorithm for a node j in the set.
    # Repeatedly traverse the parent pointer until we reach a root.
    # Implement the "rank compression" strategy by making all 
    # nodes along path from j to the root point directly to the root.
    def find(self, j):
        assert 0 <= j < self.n
        assert self.parents[j] != None, 'You are calling find on an element that is not part of the family yet. Please call make_set first.'
        # your code here
        if j != self.parents[j]:
            self.parents[j] = self.find(self.parents[j])
        return self.parents[j]        
        
    
    # Function : union
    # Compute union of j1 and j2
    # First do a find to get to the representatives of j1 and j2.
    # If they are not the same, then 
    #  implement union using the rank strategy I.e, lower rank root becomes
    #  child of the higher ranked root.
    #  break ties by making the first argument j1's root the parent.
    def union(self, j1, j2):
        assert 0 <= j1 < self.n
        assert 0 <= j2 < self.n
        assert self.parents[j1] != None
        assert self.parents[j2] != None
        # your code here
        x, y = self.find(j1), self.find(j2)
        if x != y:
            if self.get_rank(x) > self.get_rank(y):
                self.parents[y] = x
            else:
                self.parents[x] = y
                self.rank[y] += 1

In [2]:
d = DisjointForests(10)
for i in range(10):
    d.make_set(i)

for i in range(10):
    assert d.find(i) == i, f'Failed: Find on {i} must return {i} back'
    
d.union(0,1)
d.union(2,3)
assert(d.find(0) == d.find(1)), '0 and 1 have been union-ed together'
assert(d.find(2) == d.find(3)), '2 and 3 have been union-ed together'
assert(d.find(0) != d.find(3)), '0 and 3 should be in  different trees'
assert((d.get_rank(0) == 2 and d.get_rank(1) == 1) or 
       (d.get_rank(1) == 2 and d.get_rank(0) == 1)), 'one of the nodes 0 or 1 must have rank 2'

assert((d.get_rank(2) == 2 and d.get_rank(3) == 1) or 
       (d.get_rank(3) == 2 and d.get_rank(2) == 1)), 'one of the nodes 2 or 3 must have rank 2'


d.union(3,4)
assert(d.find(2) == d.find(4)), '2 and 4 must be in the same set in the family.'

d.union(5,7)
d.union(6,8)
d.union(3,7)
d.union(0,6)

assert(d.find(6) == d.find(1)), '1 and 6 must be in the same set in the family'
assert(d.find(7) == d.find(4)), '7 and 4 must be in the same set in the family'
print('All tests passed!')

All tests passed!


## Problem 2

We will now explore finding maximal strongly connected components of an undirected graph using union find data structures. 
The undirected graph just consists of a list of edges with weights.
  - We will associate a non-negative weight $w_{i,j}$ for each undirected edge $(i,j)$.
  - We associate some extra data with vertices that will come in handy later.
  
Please examine the code for undirected graph data structures carefully.

In [3]:
class UndirectedGraph:
    
    # n is the number of vertices
    # we will label the vertices from 0 to self.n -1 
    # We simply store the edges in a list.
    def __init__(self, n):
        assert n >= 1, 'You are creating an empty graph -- disallowed'
        self.n = n
        self.edges = []
        self.vertex_data = [None]*self.n
       
        
    def set_vertex_data(self, j, dat):
        assert 0 <= j < self.n
        self.vertex_data[j] = dat
        
    def get_vertex_data(self, j):
        assert 0 <= j < self.n
        return self.vertex_data[j] 
        
    def add_edge(self, i, j, wij):
        assert 0 <= i < self.n
        assert 0 <= j < self.n
        assert i != j
        # Make sure to add edge from i to j with weight wij
        self.edges.append((i, j, wij))
        
    def sort_edges(self):
        # sort edges in ascending order of weights.
        self.edges = sorted(self.edges, key=lambda edg_data: edg_data[2])

#### 2A: Use union-find data-structures to compute strongly connected components

We have previously seen  how to use DFS to find maximal strongly connected components with a small twist.

  - We will consider only those edges $(i,j)$ whose weights are less than or equal to a threshold $W$ provided by the user.
  - Edges with weights above this threshold are not considered.
  
Design an algorithm to compute all the maximal strongly connected components for all edges with threshold $W$ using the union-find data structure. What is the running time of your algorithm. 

__First make a set from each node. Then for each edge we make a union of their nodes (sets) if their parents are not the same and if the weight is less than or equal to W__

The code below computes strongly connected components.

In [4]:
def compute_scc(g, W):
    # create a disjoint forest with as many elements as number of vertices
    # Next compute the strongly connected components using the disjoint forest data structure
    d = DisjointForests(g.n)
    # your code here
    for i in range(g.n):
        d.make_set(i)
    for (i, j, wij) in g.edges:
        if wij <= W:
            if d.find(i) != d.find(j):
                d.union(i, j)
    
    # extract a set of sets from d
    return d.dictionary_of_sets()    

In [5]:
g3 = UndirectedGraph(8)
g3.add_edge(0,1,0.5)
g3.add_edge(0,2,1.0)
g3.add_edge(0,4,0.5)
g3.add_edge(2,3,1.5)
g3.add_edge(2,4,2.0)
g3.add_edge(3,4,1.5)
g3.add_edge(5,6,2.0)
g3.add_edge(5,7,2.0)
res = compute_scc(g3, 2.0)
print('SCCs with threshold 2.0 computed by the code are:')
assert len(res) == 2, f'Expected 2 SCCs but got {len(res)}'
for (k, s) in res.items():
    print(s)
    
# Let us check that your code returns what we expect.
for (k, s) in res.items():
    if (k in [0,1,2,3,4]):
        assert (s == set([0,1,2,3,4])), '{0,1,2,3,4} should be an SCC'
    if (k in [5,6,7]):
        assert (s == set([5,6,7])), '{5,6,7} should be an SCC'

        
# Let us check that the thresholding works
print('SCCs with threshold 1.5')
res2 = compute_scc(g3, 1.5) # This cutsoff edges 2,4 and 5, 6, 7
for (k, s) in res2.items():
    print(s)
assert len(res2) == 4, f'Expected 4 SCCs but got {len(res2)}'

for (k, s) in res2.items():
    if k in [0,1,2,3,4]:
        assert (s == set([0,1,2,3,4])), '{0,1,2,3,4} should be an SCC'
    if k in [5]:
        assert s == set([5]), '{5} should be an SCC with just a single node.'
    if k in [6]:
        assert s == set([6]), '{6} should be an SCC with just a single node.'
    if k in [7]:
        assert s == set([7]), '{7} should be an SCC with just a single node.'
        
print('All tests passed!')

SCCs with threshold 2.0 computed by the code are:
{0, 1, 2, 3, 4}
{5, 6, 7}
SCCs with threshold 1.5
{0, 1, 2, 3, 4}
{5}
{6}
{7}
All tests passed!


#### 2B: Compute Minimum Spanning Tree 

We will now compute the MST of a given undirected weighted graph using Kruskal's algorithm. 
The code below uses a disjoint set forest data structure for implementing Kruskal's algorithm.

The code simply returns a list of edges with edge weights as a tuple $(i,j, wij)$ that are part of the MST along with the total weight of the MST.

In [6]:
def compute_mst(g):
    # return a tuple of two items
    #   1. list of edges (i,j) that are part of the MST
    #   2. sum of MST edge weights.
    d = DisjointForests(g.n)
    mst_edges = []
    g.sort_edges()

    for s in range(g.n):
        d.make_set(s)
    for (u, v, wij) in g.edges:
        if d.find(u) != d.find(v):
            mst_edges.append((u, v, wij))
            d.union(u, v)
    mst_weight = sum(wij for u, v, wij in mst_edges)
    return (mst_edges, mst_weight)

In [7]:
g3 = UndirectedGraph(8)
g3.add_edge(0,1,0.5)
g3.add_edge(0,2,1.0)
g3.add_edge(0,4,0.5)
g3.add_edge(2,3,1.5)
g3.add_edge(2,4,2.0)
g3.add_edge(3,4,1.5)
g3.add_edge(5,6,2.0)
g3.add_edge(5,7,2.0)
g3.add_edge(3,5,2.0)

(mst_edges, mst_weight) = compute_mst(g3)
print('The code computed MST: ')
for (i,j,wij) in mst_edges:
    print(f'\t {(i,j)} weight {wij}')
print(f'Total edge weight: {mst_weight}')

assert mst_weight == 9.5, 'Optimal MST weight is expected to be 9.5'

assert (0,1,0.5) in mst_edges
assert (0,2,1.0) in mst_edges
assert (0,4,0.5) in mst_edges
assert (5,6,2.0) in mst_edges
assert (5,7,2.0) in mst_edges
assert (3,5,2.0) in mst_edges
assert (2,3, 1.5) in mst_edges or (3,4, 1.5) in mst_edges

print('All tests passed!')

The code computed MST: 
	 (0, 1) weight 0.5
	 (0, 4) weight 0.5
	 (0, 2) weight 1.0
	 (2, 3) weight 1.5
	 (5, 6) weight 2.0
	 (5, 7) weight 2.0
	 (3, 5) weight 2.0
Total edge weight: 9.5
All tests passed!
